In [1]:
from sponge import Sponge
from sponge import Molecule
from sponge import ForceField
from sponge import UpdaterMD
from sponge import WithEnergyCell
from sponge.function import VelocityGenerator
from sponge.callback import WriteH5MD, RunInfo
from sponge.optimizer import SteepestDescent
from mindspore import context

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')
system.set_pbc_box([0.4, 0.4, 0.4])

Parameter (name=pbc_box, shape=(1, 3), dtype=Float32, requires_grad=True)

如果定义了周期性的盒子，那么还可以使用repeat_box功能来扩展/复制系统，比如可以这样将系统复制1000份

In [4]:
system.repeat_box([10, 10, 10])

Molecule<>

得到的分子系统的示意图如下所示：
![](../../docs/tutorial_b05.png)

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = SteepestDescent(system.trainable_params(), 1e-6)

In [7]:
sim = WithEnergyCell(system, potential, cutoff=1.0)
md = Sponge(sim, optimizer=opt)

[MindSPONGE] Calculate the neighbour list using the inter-atomic distances.


[WARNING] KERNEL(16275,7fb231e5f500,python):2023-06-30-16:30:32.093.915 [mindspore/ccsrc/plugin/device/gpu/kernel/gpu_kernel.cc:40] CheckDeviceSm] It is recommended to use devices with a computing capacity >= 7, but the current device's computing capacity is 6


In [8]:
run_info = RunInfo(10)
md.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-06-30 16:30:34
[MindSPONGE] Step: 0, E_pot: -7163.629
[MindSPONGE] Step: 10, E_pot: -7715.919
[MindSPONGE] Step: 20, E_pot: -7753.052
[MindSPONGE] Step: 30, E_pot: -7755.6426
[MindSPONGE] Step: 40, E_pot: -7755.8687
[MindSPONGE] Step: 50, E_pot: -7755.8633
[MindSPONGE] Step: 60, E_pot: -7756.2056
[MindSPONGE] Step: 70, E_pot: -7756.234
[MindSPONGE] Step: 80, E_pot: -7756.2017
[MindSPONGE] Step: 90, E_pot: -7756.1987
[MindSPONGE] Finished simulation at 2023-06-30 16:30:42
[MindSPONGE] Simulation time: 8.48 seconds.
--------------------------------------------------------------------------------


In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system=system,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',
                pressure=1,
                barostat='berendsen',)
md = Sponge(sim, optimizer=opt)

In [11]:
md.change_optimizer(opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b05.h5md', save_freq=10)

In [13]:
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-06-30 16:30:49
[MindSPONGE] Step: 0, E_pot: -7756.2026, E_kin: 11063.09, E_tot: 3306.8872, Temperature: 295.7839, Pressure: 608.6257, Volume: 64.0
[MindSPONGE] Step: 10, E_pot: -7417.774, E_kin: 10621.939, E_tot: 3204.1655, Temperature: 283.98926, Pressure: 137.68718, Volume: 63.98105
[MindSPONGE] Step: 20, E_pot: -7861.0137, E_kin: 11176.844, E_tot: 3315.83, Temperature: 298.82523, Pressure: -215.94531, Volume: 63.95007
[MindSPONGE] Step: 30, E_pot: -9193.785, E_kin: 12348.748, E_tot: 3154.963, Temperature: 330.15738, Pressure: -1186.9487, Volume: 63.89784
[MindSPONGE] Step: 40, E_pot: -10421.905, E_kin: 13303.73, E_tot: 2881.8252, Temperature: 355.6899, Pressure: -1905.3464, Volume: 63.82028
[MindSPONGE] Step: 50, E_pot: -10846.889, E_kin: 13384.538, E_tot: 2537.6494, Temperature: 357.8504, Pressure: -2145.1445, Volume: 63.725357
[MindSPONGE] Step: 60, E_pot: -11004.227, E_kin: 13101.414, E_tot: 2097.1875, Temperature: 350.28076, Pressure: -260